In [3]:
import os
# ptb 모듈 경로 추가
import sys
sys.path.append('C:/Users/KimDongyoung/Desktop/Github/my_git/mygit/DEEPLEARNING/밑바닥부터시작하는딥러닝2')

In [4]:
from sequence import load_data, get_vocab
import os
import numpy as np
from common.time_layers import TimeEmbedding, TimeAffine, TimeSoftmaxWithLoss
from common.base_model import BaseModel
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import eval_seq2seq

In [5]:
(x_train, t_train), (x_test, t_test) = load_data('addition.txt', seed=1984)
char_to_id, id_to_char = get_vocab()

# x_train, t_train에는 문자 ID의 시퀀스가 저장되어 있다.
print(x_train.shape, t_train.shape)  # (45000, 7) (45000, 5) -> 45000개의 데이터가 있고, x_train 데이터에는 7개의 문자로 이루어져 있다.
print(x_test.shape, t_test.shape)

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)


In [6]:
print(x_train[0]) # 0~9, +, ' '의 문자 ID로 이루어진 7개의 문자로 이루어진 문제
print(''.join([id_to_char[c] for c in x_train[0]])) # 각 문자 ID에 해당하는 문자를 반환, 문제
print(''.join([id_to_char[c] for c in t_train[0]])) # 각 문자 ID에 해당하는 문자를 반환, 정답

[ 3  0  2  0  0 11  5]
71+118 
_189 


### encoder, decoder 구현 

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [8]:
class LSTM:
  def __init__(self,Wx,Wh,b):
    self.params = [Wx,Wh,b] # 가중치에는 4개분의 가중치가 담겨 있다. input gate, forget gate, output gate, block input의 가중치 4개
    self.grads = [np.zeros_like(Wx),np.zeros_like(Wh),np.zeros_like(b)] # 기울기를 저장하는 grads 인스턴스 변수 초기화
    self.cache = None       # 순전파시 중간 결과를 담을 cache변수로 역전파 계산에 사용하는 인스턴스 변수 cache를 초기화
  
  # 순전파
  def forward(self, x, h_prev, c_prev): # x는 입력 데이터, h_prev는 이전 시각의 은닉 상태, c_prev는 이전 시각의 기억 셀
    Wx, Wh, b = self.params
    N, H = h_prev.shape

    A = np.matmul(x, Wx) + np.matmul(h_prev, Wh) + b  # 4개의 가중치를 모아 아핀 변환을 수행하여 A를 계산

    f = A[:, :H]
    g = A[:, H:2*H]
    i = A[:, 2*H:3*H]
    o = A[:, 3*H:]

    f = sigmoid(f)                # forget gate
    g = np.tanh(g)                # 새로운 정보를 기억하기 위한 input
    i = sigmoid(i)                # input gate
    o = sigmoid(o)                # output gate

    c_next = f * c_prev + g * i   # 기억 셀(memory cell) 계산, c_next = f * c_prev + g * i
    h_next = o * np.tanh(c_next)  # 은닉 상태 계산, h_next = o * tanh(c_next)

    self.cache = (x, h_prev, c_prev, i, f, g, o, c_next) # cache에 순전파시 중간 결과(입력 데이터, 이전 시각의 은닉 상태, 이전 시각의 기억 셀, 4개의 gate, 기억 셀)를 저장
    return h_next, c_next                                # 은닉 상태(hidden state)와 기억 셀(memory cell)을 반환
  
  # 역전파
  def backward(self, dh_next, dc_next): # 상류에서 전해지는 기울기 dh_next와 dc_next를 받아 하류로 기울기를 전달
    Wx, Wh, b = self.params
    x, h_prev, c_prev, i, f, g, o, c_next = self.cache

    tanh_c_next = np.tanh(c_next)

    ds = dc_next + (dh_next * o) * (1 - tanh_c_next ** 2)  # ds는 tanh(c_next)의 역전파에서 내려온 기울기와 상류에서 전해지는 기울기 dh_next * o를 더한 값

    dc_prev = ds * f # 이전 시각 기억 셀(c_prev)의 기울기 dc_prev 계산, ds에 forget gate 출력 결과인 f를 곱함

    di = ds * g                 # di는 상류에서 전해지는 기울기 ds와 block input g를 곱한 값
    df = ds * c_prev            # df는 상류에서 전해지는 기울기 ds와 이전 시각 기억 셀 c_prev를 곱한 값
    do = dh_next * tanh_c_next  # do는 상류에서 전해지는 기울기 dh_next와 tanh(c_next)를 곱한 값
    dg = ds * i                 # dg는 상류에서 전해지는 기울기 ds와 input gate i를 곱한 값

    di *= i * (1 - i)           # 시그모이드 함수 기울기를 di에 곱하고 하류로 전달
    df *= f * (1 - f)           # 시그모이드 함수 기울기를 df에 곱하고 하류로 전달
    do *= o * (1 - o)           # 시그모이드 함수 기울기를 do에 곱하고 하류로 전달
    dg *= (1 - g ** 2)          # tanh 함수 기울기를 dg에 곱하고 하류로 전달

    dA = np.hstack((df, dg, di, do)) # 4개의 gate에 대한 기울기를 열 방향으로 합치기

    dWh = np.dot(h_prev.T, dA)  # dA와 h_prev의 내적을 구해 Wh에 대한 기울기 dWh를 구함
    dWx = np.dot(x.T, dA)       # dA와 x의 내적을 구해 Wx에 대한 기울기 dWx를 구함
    db = dA.sum(axis=0)         # dA의 각 행을 다 더해 db를 구함
    
    # dWx, dWh, db를 각각 grads의 0, 1, 2번째 인덱스에 저장, 4개(input gate, forget gate, output gate, block input)의 가중치에 대한 기울기를 저장
    self.grads[0][...] = dWx        
    self.grads[1][...] = dWh
    self.grads[2][...] = db

    dx = np.dot(dA, Wx.T)       # dx는 dA와 Wx의 내적을 구해 구함
    dh_prev = np.dot(dA, Wh.T)  # dh_prev는 dA와 Wh의 내적을 구해 구함

    return dx, dh_prev, dc_prev

In [9]:
# T개 분의 시계열 데이터를 한꺼번에 처리하는 TimeLSTM 계층
class TimeLSTM:
  def __init__(self, Wx, Wh, b , stateful=False):
    self.params = [Wx, Wh, b]
    self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.layers = None

    self.h, self.c = None, None   # hidden state, cell state
    self.dh = None
    self.stateful = stateful
  
  def forward(self, xs): # xs는 T개 분의 시계열 데이터를 하나로 모은 것
    Wx, Wh, b = self.params
    N, T, D = xs.shape
    H = Wh.shape[0]

    self.layers = []
    hs = np.empty((N, T, H), dtype='f')     # 출력값을 저장할 변수 hs를 초기화

    if not self.stateful or self.h is None:
      self.h = np.zeros((N, H), dtype='f')
    if not self.stateful or self.c is None:
      self.c = np.zeros((N, H), dtype='f')
    
    for t in range(T): # 반복문을 사용해 T개 분의 시계열 데이터를 한꺼번에 처리
      layer = LSTM(*self.params)
      self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c) # LSTM 계층의 forward 메서드를 호출해 은닉 상태 h와 기억 셀 c를 입력값으로 받아 은닉 상태 h와 기억 셀 c를 갱신
      hs[:, t, :] = self.h # 갱신한 은닉 상태 h를 hs에 저장

      self.layers.append(layer)
    
    return hs # hs는 은닉 상태의 시계열 데이터를 반환
  
  def backward(self, dhs):
    Wx, Wh, b = self.params
    N, T, H = dhs.shape
    D = Wx.shape[0]

    dxs = np.empty((N, T, D), dtype='f')
    dh, dc = 0, 0

    grads = [0, 0, 0]
    for t in reversed(range(T)):
      layer = self.layers[t]
      dx, dh, dc = layer.backward(dhs[:, t, :] + dh, dc)
      dxs[:, t, :] = dx
      for i, grad in enumerate(layer.grads):
        grads[i] += grad
    
    for i, grad in enumerate(grads):
      self.grads[i][...] = grad
    self.dh = dh
    return dxs
  
  def set_state(self, h, c=None): # 마지막 LSTM 계층의 은닉 상태와 기억 셀을 h와 c로 설정
    self.h, self.c = h, c
  
  # 상태 초기화
  def reset_state(self):
    self.h, self.c = None, None

In [10]:
# encoder
class Encoder:
  def __init__(self, vocab_size, wordvec_size, hidden_size): # vocab_size는 어휘 수, wordvec_size는 단어 벡터의 차원 수, hidden_size는 LSTM 계층의 은닉 상태 벡터의 차원 수
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn # 랜덤값 생성

    # 가중치 초기화
    embed_W = (rn(V, D) / 100).astype('f')            # 단어 임베딩 계층의 가중치
    lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f') # 입력값 x와 곱해지는 가중치 Wx,4개의 가중치를 모아 아핀 변환을 수행하기 위한 가중치
    lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f') # 이전 시각의 은닉 상태 h와 곱해지는 가중치 Wh, 4개의 가중치를 모아 아핀 변환을 수행하기 위한 가중치
    lstm_b = np.zeros(4 * H).astype('f')

    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False) # stateful은 은닉 상태를 유지할지 여부를 설정하는 인수

    self.params = self.embed.params + self.lstm.params
    self.grads = self.embed.grads + self.lstm.grads
    self.hs = None
  
  # 순전파
  def forward(self, xs): # xs는 단어 ID의 시퀀스 데이터
    xs = self.embed.forward(xs)
    hs = self.lstm.forward(xs)
    self.hs = hs  # hs는 LSTM 계층의 은닉 상태 벡터들을 저장
    return hs 
  
  # 역전파
  def backward(self, dh): # dh는 LSTM 계층의 마지막 시각의 은닉 상태의 기울기로 decoder에서 전해진 기울기이다. 
    dhs = np.zeros_like(self.hs) # hs와 같은 형상의 0으로 채워진 배열 dhs를 생성
    dhs[:, -1, :] = dh           # dh를 dhs의 해당 위치에 저장
    
    dout = self.lstm.backward(dhs)
    dout = self.embed.backward(dout)
    return dout
    

In [11]:
# decoder
class decoder:
  def __init__(self, vocab_size, wordvec_size, hidden_size): # vocab_size는 어휘 수, wordvec_size는 단어 벡터의 차원 수, hidden_size는 LSTM 계층의 은닉 상태 벡터의 차원 수
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn # 랜덤값 생성

    # 가중치 초기화
    embed_W = (rn(V, D) / 100).astype('f')
    
    lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f') # 입력값 x와 곱해지는 가중치 Wx,4개의 가중치를 모아 아핀 변환을 수행하기 위한 가중치
    lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f') # 이전 시각의 은닉 상태 h와 곱해지는 가중치 Wh, 4개의 가중치를 모아 아핀 변환을 수행하기 위한 가중치
    lstm_b = np.zeros(4 * H).astype('f')
    
    affine_W = (rn(H, V) / np.sqrt(H)).astype('f') # Affine 계층의 가중치 초기화
    affine_b = np.zeros(V).astype('f')             # Affine 계층의 편향 초기화
    
    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
    self.affine = TimeAffine(affine_W, affine_b)
    
    self.params, self.grads = [], []  # params와 grads에는 각 계층의 가중치와 편향을 모두 모은 리스트를 저장
    # 각 계층의 가중치와 편향을 params와 grads에 추가
    for layer in (self.embed, self.lstm, self.affine):
      self.params += layer.params
      self.grads += layer.grads
      
  # 순전파
  def forward(self, xs, h): # xs는 단어 ID의 시퀀스 데이터, h는 encoder에서 출력된 마지막 은닉 상태
    self.lstm.set_state(h)  # LSTM 계층의 은닉 상태를 encoder에서 출력된 마지막 은닉 상태로 설정
    
    out = self.embed.forward(xs)
    out = self.lstm.forward(out)
    score = self.affine.forward(out)
    return score
  
  # 역전파
  def backward(self, dscore): # dscore는 Softmax with Loss 계층의 역전파에서 전해지는 기울기
    dout = self.affine.backward(dscore)
    dout = self.lstm.backward(dout)
    dout = self.embed.backward(dout)
    dh = self.lstm.dh # decoder의 LSTM 계층의 은닉 상태의 기울기를 추출
    return dh
  
  # generate
  def generate(self, h, start_id, sample_size): # h는 encoder에서 출력된 마지막 은닉 상태, start_id는 최초로 주는 단어 ID, sample_size는 생성하는 단어의 수
    sampled = []
    sample_id = start_id
    self.lstm.set_state(h)
    
    for _ in range(sample_size):
      x = np.array(sample_id).reshape((1, 1))
      out = self.embed.forward(x)
      out = self.lstm.forward(out)
      score = self.affine.forward(out)
      
      sample_id = np.argmax(score.flatten()) # Affine 계층이 출력하는 점수가 가장 높은 문자의 ID를 sample_id로 선택
      sampled.append(int(sample_id))         # sample_id를 sampled에 추가

### seq2seq 계층 구현  

In [12]:
class Seq2seq(BaseModel):
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size # vocab_size는 어휘 수, wordvec_size는 단어 벡터의 차원 수, hidden_size는 LSTM 계층의 은닉 상태 벡터의 차원 수
    self.encoder = Encoder(V, D, H)
    self.decoder = decoder(V, D, H)
    self.softmax = TimeSoftmaxWithLoss()
    
    self.params = self.encoder.params + self.decoder.params
    self.grads = self.encoder.grads + self.decoder.grads
  
  # 순전파
  def forward(self, xs, ts): # xs는 입력 데이터, ts는 정답 레이블
    decoder_xs, decoder_ts = ts[:, :-1], ts[:, 1:]
    
    h = self.encoder.forward(xs)                    # encoder의 forward 메서드를 호출해 은닉 상태 h를 구함
    score = self.decoder.forward(decoder_xs, h)     # decoder의 forward 메서드를 호출해 점수를 구함
    loss = self.softmax.forward(score, decoder_ts)  # score와 decoder의 정답 레이블을 사용해 Softmax with Loss 계층의 forward 메서드를 호출해 손실을 구함
    return loss
  
  # 역전파
  def backward(self, dout=1):                       
    dout = self.softmax.backward(dout)              # Softmax with Loss 계층의 backward 메서드를 호출해 Softmax 계층의 기울기를 구함
    dh = self.decoder.backward(dout)                # decoder의 backward 메서드를 호출해 decoder의 LSTM 계층의 은닉 상태의 기울기를 구함
    dout = self.encoder.backward(dh)                # encoder의 backward 메서드를 호출해 encoder의 embedding 계층과 LSTM 계층의 기울기를 구함
    return dout
  
  # generate
  def generate(self, xs, start_id, sample_size):
    h = self.encoder.forward(xs)
    sampled = self.decoder.generate(h, start_id, sample_size)
    return sampled


### seq2seq 평가   

In [13]:
(x_train, t_train), (x_test, t_test) = load_data('addition.txt', seed=1984)
char_to_id, id_to_char = get_vocab()

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# 모델, 옵티마이저, 트레이너 생성
model = Seq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [14]:
acc_list = [] # 정확도를 저장할 리스트

for epoch in range(max_epoch):
  trainer.fit(x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
  correct_num = 0
  
  for i in range(len(x_test)):
    question, correct = x_test[[i]], t_test[[i]]
    verbose = i < 10
    correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose)
  
  acc = float(correct_num) / len(x_test)
  acc_list.append(acc)
  print('정확도 %.3f%%' % (acc * 100))

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 2.56
| 에폭 1 |  반복 21 / 351 | 시간 0[s] | 손실 2.53
| 에폭 1 |  반복 41 / 351 | 시간 0[s] | 손실 2.30
| 에폭 1 |  반복 61 / 351 | 시간 1[s] | 손실 2.14
| 에폭 1 |  반복 81 / 351 | 시간 1[s] | 손실 2.06
| 에폭 1 |  반복 101 / 351 | 시간 2[s] | 손실 1.99
| 에폭 1 |  반복 121 / 351 | 시간 2[s] | 손실 1.95
| 에폭 1 |  반복 141 / 351 | 시간 3[s] | 손실 1.93
| 에폭 1 |  반복 161 / 351 | 시간 3[s] | 손실 1.93
| 에폭 1 |  반복 181 / 351 | 시간 3[s] | 손실 1.92
| 에폭 1 |  반복 201 / 351 | 시간 4[s] | 손실 1.90
| 에폭 1 |  반복 221 / 351 | 시간 4[s] | 손실 1.89
| 에폭 1 |  반복 241 / 351 | 시간 5[s] | 손실 1.89
| 에폭 1 |  반복 261 / 351 | 시간 5[s] | 손실 1.87
| 에폭 1 |  반복 281 / 351 | 시간 5[s] | 손실 1.88
| 에폭 1 |  반복 301 / 351 | 시간 6[s] | 손실 1.86
| 에폭 1 |  반복 321 / 351 | 시간 6[s] | 손실 1.87
| 에폭 1 |  반복 341 / 351 | 시간 7[s] | 손실 1.85


TypeError: 'NoneType' object is not iterable

In [35]:
guess = model.predict(question)
print(guess)

AttributeError: 'Seq2seq' object has no attribute 'predict'